In [1]:
from exp.utils import *
from exp.models import *
from exp.losses import *
from fastai.vision.all import *

In [2]:
lr = 1e-2
bs = (128, 64, 32)
epochs = 100
image_sizes = (64, 128, 244)
device = get_device()

Using the GPU!


In [3]:
p_data = get_data_path()
p_train_valid = p_data/"train_valid_images"
p_train_images = p_train_valid/"train_images"
p_valid_images = p_train_valid/"valid_images"
p_test_images  = p_data/"test_images"
p_dev_images = p_data/"dev_images"

In [4]:
# Load data
label = "Cardiomegaly"
train_df, valid_df, test_df = get_dataframes(include_labels=get_labels(), 
                                             small=False)
#print(train_df.shape, valid_df.shape, test_df.shape)
train_df = get_binary_df(label, train_df)
valid_df = get_binary_df(label, valid_df)
test_df  = get_binary_df(label, test_df)

complete_df = pd.concat([train_df, valid_df, test_df])
complete_df = complete_df.drop(columns=["Follow-up #", "Patient ID", "Patient Age", "Patient Gender", 
                   "View Position", "OriginalImage[Width", "Height]", 
                   "OriginalImagePixelSpacing[x", "y]", "No Finding"])

In [5]:
def label_func(fname):
    lbl = complete_df[complete_df["Image Index"] == fname.name]["Cardiomegaly"].values[0]
    return "C" if lbl > 0 else "No C"

def splitter_func(fn):
    return fn.parent.stem == "valid_images" 

In [6]:
train_label = train_df[[label]].values
neg_weights, pos_weights = compute_class_freqs(train_label)
neg_weights, pos_weights = torch.Tensor(neg_weights), torch.Tensor(pos_weights)
weights = torch.Tensor([neg_weights, pos_weights]).to(device)
print(neg_weights, pos_weights)

tensor([0.0203]) tensor([0.9797])


In [7]:
def get_dls(bs, image_size, path=p_train_valid):
    batch_tfms = [Brightness(), 
                  Contrast(),
                  RandomErasing(p=.25, max_count=2)] 
    
    dblock = DataBlock(blocks    = (ImageBlock, CategoryBlock),
                       get_items = get_image_files,
                       get_y     = label_func,
                       splitter  = RandomSplitter(),
                       item_tfms = [Resize(image_size, 
                                          method="squish", 
                                          pad_mode="zeros")],
                       batch_tfms = batch_tfms)
    return dblock.dataloaders(path, bs=bs)

In [8]:
sched = {'lr': SchedExp(3e-3, 1e-7)}
callbacks = [
    ParamScheduler(sched),
    EarlyStoppingCallback(monitor="valid_loss",
                          patience=10),
    SaveModelCallback(monitor="valid_loss", 
                      fname="xresnet50_finetuned_v1", 
                      with_opt=True),
    ShowGraphCallback(), 
    MixUp()
]

In [10]:
# First size
dls = get_dls(bs=128, image_size=64)
model = xresnet50(n_out=dls.c)
learn = Learner(dls, 
                model, 
                loss_func=LabelSmoothingCrossEntropy(weight=weights),
                metrics=[accuracy, RocAucBinary()],
                cbs=callbacks)
learn.fit_one_cycle(30, 3e-3)

TypeError: __init__() got an unexpected keyword argument 'weight'

In [ ]:
learn.load(Path().cwd()/"models"/"xresnet50_finetuned_v1")

In [ ]:
# Second size
learn.dls = get_dls(bs=64, image_size=128)
learn.fit_one_cycle(30, 3e-3)

In [ ]:
learn.load(Path().cwd()/"models"/"xresnet50_finetuned_v1")

In [ ]:
# Final size
learn.dls = get_dls(bs=32, image_size=224)
learn.fit_one_cycle(30, 3e-3)

In [ ]:
learn.load(Path().cwd()/"models"/"xresnet50_finetuned_v1")

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(9)